In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
def get_measurements(tensor):
    # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank= calculate_slice_avg_rank(tensor)
    tensor_sparsity=calculate_sparsity(tensor.numpy())
    tensor_pictoriality=get_tensor_pictoriality(tensor)
    tensor_regularity=get_tensor_regularity(tensor)
    tensor_decomposability= (tensor_rank*tensor.shape[1]+tensor_rank*tensor.shape[2])*tensor.shape[0]
    # print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity*1.5,1-tensor_pictoriality/(tensor.numel()),1-tensor_regularity/(tensor.numel()),1-tensor_decomposability/(tensor.numel()*4)))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity,1-tensor_pictoriality,1-tensor_regularity,1-tensor_decomposability/(tensor.numel()*4)))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity,tensor_pictoriality,tensor_regularity,tensor_decomposability))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


## Low Rank

In [3]:
tensor_shape = (3,128,128)
tensor_size  = 128*128*3
rank = 3

tensor = torch.zeros(tensor_shape)
for i in range(tensor_shape[0]):
    factor_1 = torch.randn(tensor_shape[1], rank)
    factor_2 = torch.randn(tensor_shape[2], rank)
    tensor_slice = torch.einsum('ir,jr->ij', factor_1, factor_2)
    tensor[i] = tensor_slice

In [4]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.000000, Picoriality: 0.223217, Regularity: 0.201051, Decomposability: 0.988281
Rank: 3.000000, Sparsity: 0.000000, Picoriality: 0.776783, Regularity: 0.798949, Decomposability: 2304.000000
Doing JPEG compression
Ratio: 1.608311, SNR: 33.679330, MSE: 0.001185
Doing Decomposition compression
Ratio: 21.333333, SNR: 130.482101, MSE: 0.000000
Doing Regression compression
Ratio: 15.953262, SNR: 3.523766, MSE: 1.228688
Doing SketchML compression
Ratio: 4.152842, SNR: 1.675286, MSE: 1.880571


## Image-like

In [5]:
# import cv2
# import torchvision.transforms as transforms
# image = cv2.imread("./dataset/jpeg/jpeg444.jpg")
# image = cv2.resize(image,(128,128),interpolation = cv2.INTER_AREA)
# tensor = torch.from_numpy(image)
# # value_mask = abs(tensor)>20
# # tensor = tensor* value_mask
# tensor = tensor.to(torch.float32)/256
tensor = torch.load("./test_tensors/layer2/tensor_l2_34.pt")
tensor= tensor[0]

In [6]:
get_measurements(tensor)

Rank: 195.000000, Sparsity: 0.000000, Picoriality: 0.956638, Regularity: 0.360957, Decomposability: 0.531250
Rank: 195.000000, Sparsity: 0.000000, Picoriality: 0.043362, Regularity: 0.639043, Decomposability: 1297920.000000
Doing JPEG compression
Ratio: 7.365731, SNR: 27.789540, MSE: 0.002746
Doing Decomposition compression
Ratio: 0.533333, SNR: 87.082148, MSE: 0.000000
Doing Regression compression
Ratio: 15.982268, SNR: 8.253826, MSE: 0.246802
Doing SketchML compression
Ratio: 4.417941, SNR: 1.048017, MSE: 1.296969


## Regular distributed

In [7]:
import random
random.seed(5)
tensor = torch.zeros([128,128*3])

for i in range(128):
    a = random.random()
    b = random.random()*2
    c = random.random()
    for j in range(128*3):
        index = j /(128*3)
        if random.random()>0.7:
            tensor[i][j] = a*index -b*index*index+c*index*index*index+ random.random()/10
tensor = tensor.reshape([128,128,3])

In [8]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.699951, Picoriality: 0.547976, Regularity: 0.660734, Decomposability: 0.744141
Rank: 3.000000, Sparsity: 0.699951, Picoriality: 0.452024, Regularity: 0.339266, Decomposability: 50304.000000
Doing JPEG compression
Ratio: 4.988911, SNR: 32.047489, MSE: 0.000022
Doing Decomposition compression
Ratio: 0.977099, SNR: 138.920984, MSE: 0.000000
Doing Regression compression
Ratio: 14.222222, SNR: 20.952475, MSE: 0.000278
Doing SketchML compression
Ratio: 12.011730, SNR: 3.230087, MSE: 0.016462


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/sketchml.py:185: RuntimeWarning: Mean of empty slice.
  bucket_means[k] = values[bucket_mask].mean()
/home/rex/archiconda3/envs/pytorch/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


## Random tensor

In [9]:
import random
random.seed(5)
tensor = torch.zeros([128,128*3])
for i in range(128):
    for j in range(128*3):
        if random.random()>0.8:
            tensor[i][j] = random.gauss(1, 0.3)
tensor = tensor.reshape([128,128,3])

In [10]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.799744, Picoriality: 0.327023, Regularity: 0.732797, Decomposability: 0.744141
Rank: 3.000000, Sparsity: 0.799744, Picoriality: 0.672977, Regularity: 0.267203, Decomposability: 50304.000000
Doing JPEG compression
Ratio: 2.959538, SNR: 18.256607, MSE: 0.003253
Doing Decomposition compression
Ratio: 0.977099, SNR: 136.591005, MSE: 0.000000
Doing Regression compression
Ratio: 14.222222, SNR: 11.030800, MSE: 0.017175
Doing SketchML compression
Ratio: 16.451176, SNR: 12.256204, MSE: 0.012952


In [11]:
test_tensor = torch.load("./test_tensors/layer8/tensor_l8_0.pt")

In [12]:
get_tensor_entropy(test_tensor)/ tensor.numel()

tensor(0.0544)